# Necessary library installations

In [1]:
# Installation of AutoML SOTA library and dask
!pip install autogluon
!pip install dask

# Some simple Feature Engineering and Data Cleaning

In [ ]:
import numpy as np
import pandas as pd
import os
from autogluon.tabular  import TabularDataset, TabularPredictor


X_train = pd.read_csv('X_train_Hi5.csv')


In [31]:
X_test = pd.read_csv('X_test_Hi5.csv',low_memory=False)

In [32]:
Final_predi=pd.read_csv("y_test_Hi5.csv", delimiter=',')
 # This is the file that contains the predictions of the models that we have trained in the previous steps

In [33]:
y_test=Final_predi['piezo_groundwater_level_category'].to_numpy()

In [34]:
# Remove columns with a missing rate higher than 50%
missing_rates = X_train.isnull().mean()
columns_to_keep = missing_rates[missing_rates <= 0.5].index

X_train=X_train[columns_to_keep[1:]]
X_test = X_test[columns_to_keep[1:-1]]

In [35]:
# Convert INSEE columns to float64 data type
insee_columns = X_train.filter(regex='^insee_').columns
X_train[insee_columns] = X_train[insee_columns].apply(pd.to_numeric, errors='coerce')
X_test[insee_columns] = X_test[insee_columns].apply(pd.to_numeric, errors='coerce')
print(X_train[insee_columns].dtypes)

insee_%_agri              float64
insee_pop_commune         float64
insee_med_living_level    float64
insee_%_ind               float64
insee_%_const             float64
dtype: object


In [36]:
non_numeric_cols = X_test.select_dtypes(include=['object', 'category']).columns
time_cols = [col for col in X_train.columns if 'date' in col.lower()]
categorical_cols = [col for col in non_numeric_cols if col not in time_cols]

In [37]:
# Define some preprocessing functions
def date(df):
    df['piezo_measurement_date'] = pd.to_datetime(df['piezo_measurement_date'])
    df['year'] = df['piezo_measurement_date'].dt.year
    df['month'] = df['piezo_measurement_date'].dt.month
    df['day'] = df['piezo_measurement_date'].dt.day
    df = df.drop(columns=['piezo_measurement_date'])
    return df

def engineer_features(df):

    for window in [7, 14]:
        df[f'temp_avg_{window}d'] = df.groupby('piezo_station_bss_code')['meteo_temperature_avg'] \
            .rolling(window=window, min_periods=1) \
            .mean() \
            .reset_index(0, drop=True)

        df[f'rain_sum_{window}d'] = df.groupby('piezo_station_bss_code')['meteo_rain_height'] \
            .rolling(window=window, min_periods=1) \
            .sum() \
            .reset_index(0, drop=True)
    df['rain_temp_interaction'] = df['meteo_rain_height'] * df['meteo_temperature_avg']
    return df

In [38]:
columns_to_drop = ['piezo_station_update_date', 'meteo_date', 'hydro_observation_date_elab']
X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

# DateTime preprocessing
X_train=date(X_train)
X_test=date(X_test)

# Preliminary Feature Engineering
X_train=engineer_features(X_train)
X_test=engineer_features(X_test)

In [39]:
non_numeric_cols = X_test.select_dtypes(include=['object', 'category']).columns
numeric_cols=X_train.select_dtypes(include=['number']).columns
time_cols = [col for col in X_train.columns if 'date' in col.lower()]
categorical_cols = [col for col in non_numeric_cols if col not in time_cols]

In [40]:
for col in numeric_cols:
    X_train[col].fillna(X_train[col].median(), inplace=True)
    X_test[col].fillna(X_test[col].median(), inplace=True)
for col in categorical_cols:
    X_train[col].fillna('nan', inplace=True)
    X_test[col].fillna('nan', inplace=True)

C:\Users\JLASSI\AppData\Local\Temp\ipykernel_3464\1619863657.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(X_train[col].median(), inplace=True)
C:\Users\JLASSI\AppData\Local\Temp\ipykernel_3464\1619863657.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


# Model Training

In [44]:
print(X_train.dtypes)


piezo_station_department_code         object
piezo_station_investigation_depth    float64
piezo_station_department_name         object
piezo_station_commune_code_insee      object
piezo_station_pe_label                object
                                      ...   
temp_avg_7d                          float64
rain_sum_7d                          float64
temp_avg_14d                         float64
rain_sum_14d                         float64
rain_temp_interaction                float64
Length: 95, dtype: object


In [45]:
import optuna
from autogluon.tabular import TabularDataset, TabularPredictor

label_column ='piezo_groundwater_level_category'

def objective(trial):
    # Suggest hyperparameters for XGBoost
    hyperparameters = {
        
        'GBM': {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        # Add additional hyperparameters if needed, based on GBM's supported parameters
            },
        'XGB': {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        },
    }

    # Train TabularPredictor
    predictor = TabularPredictor(label=label_column, eval_metric='f1_weighted')
    try:
        predictor.fit(train_data=X_train, hyperparameters=hyperparameters, verbosity=3)
        # Evaluate predictions
        predictions = predictor.predict(X_test)
        score = predictor.evaluate_predictions(y_true=X_test[label_column], y_pred=predictions)
        return -score['f1_weighted']  # Optuna minimizes, so return negative metric
    except Exception as e:
        print(f"Trial failed with exception: {e}")
        return float('inf')  # Return a high value for failed trials


# Create the Optuna study
study = optuna.create_study(direction='maximize')  # Maximize f1_weighted
study.optimize(objective, n_trials=10)



[I 2025-01-01 20:14:43,035] A new study created in memory with name: no-name-32a4e645-9630-472c-9db4-833669e472c0
No path specified. Models will be saved in: "AutogluonModels\ag-20250101_191443"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
GPU Count:          0
Memory Avail:       13.15 GB / 39.32 GB (33.4%)
Disk Space Avail:   180.19 GB / 449.47 GB (40.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support G

[0]	validation_0-mlogloss:1.56643	validation_0-_f1_weighted:-0.33302
[50]	validation_0-mlogloss:1.20327	validation_0-_f1_weighted:-0.54159
[100]	validation_0-mlogloss:1.10023	validation_0-_f1_weighted:-0.59996
[150]	validation_0-mlogloss:1.02983	validation_0-_f1_weighted:-0.63421
[177]	validation_0-mlogloss:0.99427	validation_0-_f1_weighted:-0.65219


Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_191443\models\XGBoost\model.pkl
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_191443\utils\attr\XGBoost\y_pred_proba_val.pkl
	0.6522	 = Validation score   (f1_weighted)
	401.44s	 = Training   runtime
	0.49s	 = Validation runtime
	57225.6	 = Inference  throughput (rows/s | 28304 batch size)
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_191443\models\trainer.pkl
Loading: c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_191443\utils\attr\XGBoost\y_pred_proba_val.pkl
Model configs that will be trained (in order):
	WeightedEnsemble_L2: 	{'ag_args': {'valid_base': False, 'name_bag_suffix': '', 'model_type': <class 'autogluon.core.models.greedy_ensemble.greedy_weighted_ensemble_model.GreedyWeightedEnsembleModel'>, 'priority': 0}, 'ag_args_ensemble': {'save_bag_folds': True}}
Fitting model: WeightedEnsemble_L2 ...
	Fitting WeightedEnsemble_L2 with 'num_gpus': 0, 'num_cpus': 12
Sa

Trial failed with exception: 'piezo_groundwater_level_category'


	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_192545\learner.pkl
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_192545\predictor.pkl
Beginning AutoGluon training ...
AutoGluon will save models to "c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_192545"
Train Data Rows:    2830316
Train Data Columns: 94
Label Column:       piezo_groundwater_level_category
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	5 unique label values:  ['High', 'Very High', 'Very Low', 'Low', 'Average']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quant

...

[0]	validation_0-mlogloss:1.60476	validation_0-_f1_weighted:-0.32547
[50]	validation_0-mlogloss:1.46532	validation_0-_f1_weighted:-0.39349
[100]	validation_0-mlogloss:1.40555	validation_0-_f1_weighted:-0.42005
[105]	validation_0-mlogloss:1.40098	validation_0-_f1_weighted:-0.42307


Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_223949\models\XGBoost\model.pkl
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_223949\utils\attr\XGBoost\y_pred_proba_val.pkl
	0.4231	 = Validation score   (f1_weighted)
	246.54s	 = Training   runtime
	0.33s	 = Validation runtime
	85413.4	 = Inference  throughput (rows/s | 28304 batch size)
Saving c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_223949\models\trainer.pkl
Loading: c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_223949\utils\attr\XGBoost\y_pred_proba_val.pkl
Model configs that will be trained (in order):
	WeightedEnsemble_L2: 	{'ag_args': {'valid_base': False, 'name_bag_suffix': '', 'model_type': <class 'autogluon.core.models.greedy_ensemble.greedy_weighted_ensemble_model.GreedyWeightedEnsembleModel'>, 'priority': 0}, 'ag_args_ensemble': {'save_bag_folds': True}}
Fitting model: WeightedEnsemble_L2 ...
	Fitting WeightedEnsemble_L2 with 'num_gpus': 0, 'num_cpus': 12
Sa

Trial failed with exception: 'piezo_groundwater_level_category'
Best hyperparameters: {'n_estimators': 178, 'max_depth': 6, 'learning_rate': 0.28333206115755916, 'subsample': 0.5198366253240017, 'colsample_bytree': 0.8522255121629868}
Best score: inf


	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "c:\Users\JLASSI\Downloads\HAck\AutogluonModels\ag-20250101_224716"
Train Data Rows:    2830316
Train Data Columns: 94
Label Column:       piezo_groundwater_level_category
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	5 unique label values:  ['High', 'Very High', 'Very Low', 'Low', 'Average']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 5
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGene

AssertionError: Unknown model type specified in hyperparameters: 'XGBoost'. Valid model types: ['RF', 'XT', 'KNN', 'GBM', 'CAT', 'XGB', 'NN_TORCH', 'LR', 'FASTAI', 'TRANSF', 'AG_TEXT_NN', 'AG_IMAGE_NN', 'AG_AUTOMM', 'FT_TRANSFORMER', 'TABPFN', 'TABPFNMIX', 'FASTTEXT', 'ENS_WEIGHTED', 'SIMPLE_ENS_WEIGHTED', 'IM_RULEFIT', 'IM_GREEDYTREE', 'IM_FIGS', 'IM_HSTREE', 'IM_BOOSTEDRULES', 'VW', 'DUMMY']

In [47]:
# Print the best parameters and score
print("Best hyperparameters:", study.best_params)
print("Best score:", study.best_value)



Best hyperparameters: {'n_estimators': 178, 'max_depth': 6, 'learning_rate': 0.28333206115755916, 'subsample': 0.5198366253240017, 'colsample_bytree': 0.8522255121629868}
Best score: inf


In [48]:
# Extract best hyperparameters for each model type
best_hyperparameters = {
    'XGB': {
        'n_estimators': study.best_params.get('n_estimators', 100),
        'max_depth': study.best_params.get('max_depth', 6),
        'learning_rate': study.best_params.get('learning_rate', 0.1),
        'subsample': study.best_params.get('subsample', 1.0),
        'colsample_bytree': study.best_params.get('colsample_bytree', 1.0),
    },
    'GBM': {
        'n_estimators': study.best_params.get('n_estimators', 100),
        'early_stopping_rounds': study.best_params.get('early_stopping_rounds', 20),
        'learning_rate': study.best_params.get('learning_rate', 0.1),
    },
}

# Train the final model with the best hyperparameters
final_predictor = TabularPredictor(label=label_column, eval_metric='f1_weighted')
final_predictor.fit(train_data=X_train, hyperparameters=best_hyperparameters, verbosity=3)


No path specified. Models will be saved in: "AutogluonModels\ag-20250101_225659"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       14.42 GB / 39.32 GB (36.7%)
Disk Space Avail:   148.42 GB / 449.47 GB (33.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'        

In [49]:
predictions = final_predictor.predict(X_test)

In [52]:
accuracy_score = np.mean(y_test==predictions)

In [53]:
accuracy_score

0.5222886480543448